In [4]:
# import pandas as pd

# # Read the CSV file
# df = pd.read_csv("news_dataset.csv", encoding='latin1')

# # Check for missing values
# print(df.isnull().sum())

# # Drop rows with null values
# df = df.dropna()

# # Verify null values are dropped
# print(df.isnull().sum())

# df.to_csv("news_dataset.csv", index=False)


Label    0
News     8
dtype: int64
Label    0
News     0
dtype: int64


In [5]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import re
import pickle

# Read the CSV file
df = pd.read_csv("news_dataset.csv", encoding='latin1')

# Check for missing values
df.isnull().sum()

# Get class counts
class_count = df["Label"].value_counts()
class_count

FAKE    1871
REAL    1850
Name: Label, dtype: int64

In [6]:
df.isnull().sum()

Label    0
News     0
dtype: int64

In [7]:
# Perform stemming
porter_stem = PorterStemmer()

def stemming(content):
    con = re.sub('[^a-zA-Z]', ' ', content)
    con = con.lower()
    con = con.split()
    con = [porter_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con = ' '.join(con)
    return con

df['News'] = df['News'].apply(stemming)

# Split data into features and labels
x = df['News']
y = df['Label']

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

# Create TF-IDF vectorizer
vect = TfidfVectorizer()

# Fit and transform the training data
x_train = vect.fit_transform(x_train)

# Transform the testing data
x_test = vect.transform(x_test)

# Create and train the SVM model
model = SVC()
model.fit(x_train, y_train)

# Make predictions on the testing data
prediction = model.predict(x_test)

# Calculate the model accuracy
accuracy = model.score(x_test, y_test)
print("Accuracy:", accuracy)

# Save the vectorizer and model using pickle
pickle.dump(vect, open('vector.pkl', 'wb'))
pickle.dump(model, open('model.pkl', 'wb'))

# Load the vectorizer and model from the saved files
vector_form = pickle.load(open('vector.pkl', 'rb'))
load_model = pickle.load(open('model.pkl', 'rb'))

# Function to classify news as fake or not
def fake_news(news):
    news = stemming(news)
    input_data = [news]
    vector_form1 = vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction



Accuracy: 0.9959731543624161


In [12]:
# Test the function with a news input
result = fake_news('''Video of Rahul Gandhi refusing to do aarti''')
print(result)


['FAKE']
